In [13]:
import cPickle
from PIL import Image
import os
import sys
import numpy as np
from matplotlib.pyplot import imshow
import tensorflow as tf
import random
import datetime


def read_filemapping(filemapping_name):
    img_filenames = []
    labels = []
    with open(filemapping_name, 'r') as f:
        lines = f.readlines()
        for line in lines:
            filename, label = line.strip('\n').split('\t')
            label = int(label)
            labels.append(label)
            img_filenames.append(filename)
    return img_filenames, labels

train_img_filenames, train_labels = read_filemapping('images/train_filemapping')
test_img_filenames, test_labels = read_filemapping('images/test_filemapping')
print train_img_filenames[0:3]
print train_labels[0:3]
print test_img_filenames[0:3]
print test_labels[0:3]
print len(train_img_filenames), len(train_labels)
print len(test_img_filenames), len(test_labels)
        

['images/train_images/leptodactylus_pentadactylus_s_000004.png', 'images/train_images/camion_s_000148.png', 'images/train_images/tipper_truck_s_001250.png']
[6, 9, 9]
['images/test_images/domestic_cat_s_000907.png', 'images/test_images/hydrofoil_s_000078.png', 'images/test_images/sea_boat_s_001456.png']
[3, 8, 8]
50000 50000
10000 10000


In [8]:
filename_placeholder = tf.placeholder(tf.string, shape=[None])
labels_placeholder = tf.placeholder(tf.int32, shape=[None])

dataset = tf.data.Dataset.from_tensor_slices((filename_placeholder, labels_placeholder))

def read_img(filename, label):
    img_string = tf.read_file(filename)
    img_decoded = tf.image.decode_image(img_string)
    return img_decoded, label

batch_size = 20
dataset = dataset.map(read_img, num_parallel_calls=batch_size)
dataset = dataset.prefetch(buffer_size=2000)
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(buffer_size=2000)
dataset = dataset.repeat()
# iterator = dataset.make_one_shot_iterator()
iterator = dataset.make_initializable_iterator()
batch_imgs, batch_labels = iterator.get_next()
batch_imgs = tf.cast(batch_imgs, tf.float32)
batch_labels = tf.cast(batch_labels, tf.int64)


In [9]:
def cnn_model(x, y):
    with tf.name_scope("inputs"):
        is_training = tf.placeholder(tf.bool, name="is_training")
        x_image = tf.reshape(x, [-1, 32, 32, 3])
    with tf.name_scope("model"):
        conv1 = tf.layers.conv2d(inputs=x_image,
                                 filters=32,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size=[2,2],
                                        strides=2)
        
        conv2 = tf.layers.conv2d(inputs=pool1,
                                 filters=64,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(inputs=conv2,
                                        pool_size=[2,2],
                                        strides=2)
        
        conv3 = tf.layers.conv2d(inputs=pool2,
                                 filters=64,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling2d(inputs=conv3,
                                        pool_size=[2,2],
                                        strides=2)
        
        pool3_flatten = tf.layers.flatten(pool3)
        dense = tf.layers.dense(inputs=pool3_flatten,
                                units=128,
                                activation=tf.nn.relu)
        dropout = tf.layers.dropout(inputs=dense,
                                    rate=0.1,
                                    training=is_training)
        logits = tf.layers.dense(inputs=dropout,
                                units=10)
    return is_training, logits

In [16]:
is_training, logits = cnn_model(batch_imgs, batch_labels)
with tf.name_scope('metrics'):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=batch_labels, logits=logits))
    correct_prediction = tf.equal(tf.argmax(logits, 1), batch_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)

init = tf.global_variables_initializer()

train_steps = 500
with tf.Session() as sess:
    sess.run(init)
    sess.run(iterator.initializer,
             feed_dict = {
                 filename_placeholder: train_img_filenames,
                 labels_placeholder: train_labels
             })
    starttime = datetime.datetime.now()
    for i in range(train_steps):
        cross_entropy_val, acc_val, _ = sess.run([cross_entropy, accuracy, train_op],
                                                 feed_dict = {
                                                     is_training: True
                                                 })


        if (i+1) % 50 == 0:
            endtime = datetime.datetime.now()
            timedelta = endtime - starttime
            print '[Train] Step %4d: ce: %4.5f, acc: %4.5f [Time: %d.%d s]' % (
                i+1, cross_entropy_val, acc_val,
                timedelta.seconds, timedelta.microseconds / 1000)
            starttime = datetime.datetime.now()

[Train] Step   50: ce: 2.00812, acc: 0.35000 [Time: 19.493 s]
[Train] Step  100: ce: 2.19649, acc: 0.10000 [Time: 5.753 s]
[Train] Step  150: ce: 1.93401, acc: 0.25000 [Time: 6.34 s]
[Train] Step  200: ce: 2.03610, acc: 0.20000 [Time: 6.29 s]
[Train] Step  250: ce: 2.00014, acc: 0.35000 [Time: 6.13 s]
[Train] Step  300: ce: 1.99023, acc: 0.35000 [Time: 5.945 s]
[Train] Step  350: ce: 2.23444, acc: 0.10000 [Time: 5.546 s]
[Train] Step  400: ce: 1.71250, acc: 0.35000 [Time: 5.549 s]
[Train] Step  450: ce: 1.87933, acc: 0.15000 [Time: 4.971 s]
[Train] Step  500: ce: 1.42025, acc: 0.60000 [Time: 4.994 s]
